<a href="https://colab.research.google.com/github/somayeh1404/cod/blob/main/CNN_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d quynhlecl/lung-cancer-x-ray


!unzip Chest X-Ray Images (lung cancer).zip

!ls

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/quynhlecl/lung-cancer-x-ray
License(s): MIT
 99% 2.26G/2.28G [00:21<00:00, 241MB/s]
100% 2.28G/2.28G [00:21<00:00, 114MB/s]
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `unzip Chest X-Ray Images (lung cancer).zip'
lung-cancer-x-ray.zip  sample_data


In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
import zipfile


BATCH_SIZE = 32
DROPOUT = 0.5
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
IMG_SIZE = 224
PATIENCE = 5


train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def setup_dataset():
    base_path = '/content/lung-cancer-x-ray'
    zip_path = '/content/lung-cancer-x-ray.zip'

    if not os.path.exists(base_path):
        print("Downloading dataset...")
        os.makedirs('/content', exist_ok=True)

        # دانلود از کگل
        os.system('kaggle datasets download -d quynhlecl/lung-cancer-x-ray -p /content/')

        # اکسترکت کردن فایل
        if os.path.exists(zip_path):
            print("Extracting dataset...")
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('/content/')

            # بررسی ساختار پوشه‌ها
            extracted_paths = [
                '/content/lung-cancer-x-ray',
                '/content/Lung Cancer X-ray',
                '/content/dataset'
            ]

            for path in extracted_paths:
                if os.path.exists(path):
                    print(f"Found dataset at: {path}")
                    # اگر ساختار پوشه متفاوت است، آن را به ساختار مورد نظر تغییر می‌دهیم
                    if 'Lung Cancer X-ray' in path:
                        os.rename(path, base_path)
                    return base_path

            # اگر هیچکدام از مسیرهای فوق پیدا نشد، محتوای دایرکتوری را بررسی می‌کنیم
            print("Checking content directory structure...")
            for item in os.listdir('/content'):
                item_path = os.path.join('/content', item)
                if os.path.isdir(item_path) and ('lung' in item.lower() or 'cancer' in item.lower()):
                    print(f"Found potential dataset directory: {item_path}")
                    os.rename(item_path, base_path)
                    return base_path
        else:
            raise FileNotFoundError("Dataset zip file not found after download attempt")

    return base_path

try:
    base_path = setup_dataset()
    train_path = os.path.join(base_path, 'train')
    test_path = os.path.join(base_path, 'test')

    # بررسی وجود پوشه‌های train و test
    if not os.path.exists(train_path):
        print(f"Train path not found: {train_path}")
        print("Available directories in base path:")
        for item in os.listdir(base_path):
            print(f"  - {item}")

        # اگر ساختار متفاوت است، سعی می‌کنیم پوشه‌های مناسب را پیدا کنیم
        for item in os.listdir(base_path):
            item_path = os.path.join(base_path, item)
            if os.path.isdir(item_path):
                subitems = os.listdir(item_path)
                if 'train' in subitems or 'test' in subitems:
                    print(f"Found structured dataset in: {item_path}")
                    train_path = os.path.join(item_path, 'train')
                    test_path = os.path.join(item_path, 'test')
                    break
                elif any('train' in subitem.lower() for subitem in subitems):
                    # پیدا کردن پوشه‌های train و test
                    for subitem in subitems:
                        subitem_path = os.path.join(item_path, subitem)
                        if 'train' in subitem.lower() and os.path.isdir(subitem_path):
                            train_path = subitem_path
                        elif 'test' in subitem.lower() and os.path.isdir(subitem_path):
                            test_path = subitem_path

    print(f"Final train path: {train_path}")
    print(f"Final test path: {test_path}")

    # بارگذاری دیتاست
    train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
    test_dataset = datasets.ImageFolder(root=test_path, transform=val_transform)

    print(f"Train classes: {train_dataset.classes}")
    print(f"Test classes: {test_dataset.classes}")
    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")

except Exception as e:
    print(f"Error setting up dataset: {e}")
    print("Trying alternative dataset structure...")

    # روش جایگزین: بررسی تمام پوشه‌های ممکن
    possible_paths = [
        '/content/lung-cancer-x-ray',
        '/content/Lung Cancer X-ray',
        '/content/dataset',
        '/content'
    ]

    for path in possible_paths:
        if os.path.exists(path):
            print(f"Checking {path}...")
            for root, dirs, files in os.walk(path):
                for dir_name in dirs:
                    if 'train' in dir_name.lower():
                        train_path = os.path.join(root, dir_name)
                        print(f"Found train directory: {train_path}")
                    if 'test' in dir_name.lower():
                        test_path = os.path.join(root, dir_name)
                        print(f"Found test directory: {test_path}")

    if 'train_path' in locals() and 'test_path' in locals():
        train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
        test_dataset = datasets.ImageFolder(root=test_path, transform=val_transform)
    else:
        raise FileNotFoundError("Could not find train and test directories in any expected location")


# ادامه کد قبلی...
targets = np.array([label for _, label in train_dataset])
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(np.zeros(len(targets)), targets))

train_subset = Subset(train_dataset, train_idx)
val_subset = Subset(train_dataset, val_idx)


class_counts = np.bincount(targets[train_idx])
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float32)


train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(DROPOUT),
    nn.Linear(num_ftrs, 2)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2)


best_val_accuracy = 0
no_improve = 0

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total


    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total

    scheduler.step(val_acc)

    print(f'\nEpoch {epoch+1}/{NUM_EPOCHS}:')
    print(f'Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%')


    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            print(f'\nEarly stopping at epoch {epoch+1}')
            break


model.load_state_dict(torch.load('best_model.pth'))
model.eval()

all_labels = []
all_preds = []
test_correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

test_acc = 100 * test_correct / len(test_dataset)
print(f'\nTest Accuracy: {test_acc:.2f}%')
print('\nClassification Report:')
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

print('\nConfusion Matrix:')
print(confusion_matrix(all_labels, all_preds))

Extracting dataset...
Checking content directory structure...
Found potential dataset directory: /content/chest_xray_lung
Final train path: /content/lung-cancer-x-ray/train
Final test path: /content/lung-cancer-x-ray/test
Train classes: ['Cancer', 'NORMAL']
Test classes: ['Cancer', 'NORMAL']
Train dataset size: 5216
Test dataset size: 624


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 154MB/s]



Epoch 1/10:
Train Loss: 0.1803 | Acc: 93.17%
Val Loss: 0.2507 | Acc: 92.24%

Epoch 2/10:
Train Loss: 0.1230 | Acc: 95.78%
Val Loss: 0.1612 | Acc: 96.17%

Epoch 3/10:
Train Loss: 0.1062 | Acc: 96.09%
Val Loss: 0.0595 | Acc: 97.61%

Epoch 4/10:
Train Loss: 0.0949 | Acc: 96.38%
Val Loss: 0.0633 | Acc: 97.61%

Epoch 5/10:
Train Loss: 0.0891 | Acc: 96.62%
Val Loss: 0.0752 | Acc: 96.46%

Epoch 6/10:
Train Loss: 0.0912 | Acc: 96.50%
Val Loss: 0.1552 | Acc: 93.30%

Epoch 7/10:
Train Loss: 0.0800 | Acc: 96.88%
Val Loss: 0.0520 | Acc: 97.80%

Epoch 8/10:
Train Loss: 0.0497 | Acc: 98.08%
Val Loss: 0.0388 | Acc: 98.08%

Epoch 9/10:
Train Loss: 0.0422 | Acc: 98.54%
Val Loss: 0.0432 | Acc: 98.37%

Epoch 10/10:
Train Loss: 0.0521 | Acc: 98.08%
Val Loss: 0.0476 | Acc: 98.37%

Test Accuracy: 87.02%

Classification Report:
              precision    recall  f1-score   support

      Cancer       0.83      0.99      0.91       390
      NORMAL       0.98      0.67      0.79       234

    accuracy      

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# تنظیمات پیشرفته
class Config:
    BATCH_SIZE = 32
    DROPOUT = 0.6  # افزایش برای کاهش اورفیت
    WEIGHT_DECAY = 1e-3  # افزایش regularization
    LEARNING_RATE = 0.0005  # کاهش نرخ یادگیری
    NUM_EPOCHS = 10
    IMG_SIZE = 224
    PATIENCE = 7
    NUM_CLASSES = 2

config = Config()

# Data Augmentation قوی‌تر
train_transform = transforms.Compose([
    transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.RandomResizedCrop(config.IMG_SIZE, scale=(0.8, 1.0)),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def setup_dataset():
    base_path = '/content/lung-cancer-x-ray'
    zip_path = '/content/lung-cancer-x-ray.zip'

    if not os.path.exists(base_path):
        print("Downloading dataset...")
        os.makedirs('/content', exist_ok=True)

        # دانلود از کگل
        os.system('kaggle datasets download -d quynhlecl/lung-cancer-x-ray -p /content/')

        # اکسترکت کردن فایل
        if os.path.exists(zip_path):
            print("Extracting dataset...")
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('/content/')

            # بررسی ساختار پوشه‌ها
            extracted_paths = [
                '/content/lung-cancer-x-ray',
                '/content/Lung Cancer X-ray',
                '/content/dataset'
            ]

            for path in extracted_paths:
                if os.path.exists(path):
                    print(f"Found dataset at: {path}")
                    if 'Lung Cancer X-ray' in path:
                        os.rename(path, base_path)
                    return base_path

            print("Checking content directory structure...")
            for item in os.listdir('/content'):
                item_path = os.path.join('/content', item)
                if os.path.isdir(item_path) and ('lung' in item.lower() or 'cancer' in item.lower()):
                    print(f"Found potential dataset directory: {item_path}")
                    os.rename(item_path, base_path)
                    return base_path
        else:
            raise FileNotFoundError("Dataset zip file not found after download attempt")

    return base_path

def find_dataset_paths(base_path):
    """پیدا کردن مسیرهای train و test به صورت هوشمند"""
    train_path = os.path.join(base_path, 'train')
    test_path = os.path.join(base_path, 'test')

    if not os.path.exists(train_path):
        print("Searching for train/test directories...")
        for root, dirs, files in os.walk(base_path):
            for dir_name in dirs:
                full_path = os.path.join(root, dir_name)
                if 'train' in dir_name.lower():
                    train_path = full_path
                    print(f"Found train directory: {train_path}")
                elif 'test' in dir_name.lower():
                    test_path = full_path
                    print(f"Found test directory: {test_path}")

    return train_path, test_path

# کلاس مدل پیشرفته‌تر
class AdvancedResNet(nn.Module):
    def __init__(self, num_classes=2, dropout=0.6):
        super(AdvancedResNet, self).__init__()
        self.backbone = models.resnet18(pretrained=True)
        num_ftrs = self.backbone.fc.in_features

        # لایه‌های طبقه‌بندی پیشرفته‌تر
        self.backbone.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(num_ftrs, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout-0.1),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout-0.2),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

def calculate_class_weights(dataset):
    """محاسبه وزن کلاس‌ها برای مدیریت imbalance"""
    targets = np.array([label for _, label in dataset])
    class_counts = np.bincount(targets)
    total_samples = len(targets)

    # روش پیشرفته‌تر برای محاسبه وزن
    class_weights = total_samples / (len(class_counts) * class_counts)
    return torch.tensor(class_weights, dtype=torch.float32)

def create_weighted_sampler(dataset, indices):
    """ایجاد sampler برای مدیریت imbalance در حین آموزش"""
    targets = np.array([label for _, label in dataset])
    targets = targets[indices]
    class_counts = np.bincount(targets)
    class_weights = 1. / class_counts
    sample_weights = class_weights[targets]

    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    return sampler

def train_epoch(model, dataloader, criterion, optimizer, device):
    """آموزش یک اپوک"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(dataloader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient Clipping برای پایداری آموزش
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        pbar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Acc': f'{100 * correct / total:.2f}%'
        })

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

def validate_epoch(model, dataloader, criterion, device):
    """ارزیابی مدل"""
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc='Validation'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            # ذخیره احتمالات برای محاسبه AUC
            probs = torch.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss = val_loss / len(dataloader)
    val_acc = 100 * val_correct / val_total

    # محاسبه AUC
    all_probs = np.array(all_probs)
    all_labels = np.array(all_labels)
    auc = roc_auc_score(all_labels, all_probs[:, 1])

    return val_loss, val_acc, auc

def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    """پلات کردن تاریخچه آموزش"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # پلات loss
    ax1.plot(train_losses, label='Train Loss')
    ax1.plot(val_losses, label='Val Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)

    # پلات accuracy
    ax2.plot(train_accs, label='Train Accuracy')
    ax2.plot(val_accs, label='Val Accuracy')
    ax2.set_title('Training and Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

def main():
    print("🚀 Starting Advanced Lung Cancer Detection Training...")

    # تنظیم device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # تنظیم دیتاست
    try:
        base_path = setup_dataset()
        train_path, test_path = find_dataset_paths(base_path)

        print(f"Train path: {train_path}")
        print(f"Test path: {test_path}")

        # بارگذاری دیتاست
        train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
        test_dataset = datasets.ImageFolder(root=test_path, transform=val_transform)

        print(f"Train classes: {train_dataset.classes}")
        print(f"Test classes: {test_dataset.classes}")
        print(f"Train dataset size: {len(train_dataset)}")
        print(f"Test dataset size: {len(test_dataset)}")

        # نمایش توزیع کلاس‌ها
        train_targets = np.array([label for _, label in train_dataset])
        class_counts = np.bincount(train_targets)
        print(f"Class distribution in train: {dict(zip(train_dataset.classes, class_counts))}")

    except Exception as e:
        print(f"Error in dataset setup: {e}")
        return

    # تقسیم داده به train/validation
    targets = np.array([label for _, label in train_dataset])
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idx, val_idx = next(sss.split(np.zeros(len(targets)), targets))

    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    # محاسبه وزن کلاس‌ها
    class_weights = calculate_class_weights(train_dataset)
    print(f"Class weights: {class_weights}")

    # ایجاد weighted sampler
    train_sampler = create_weighted_sampler(train_dataset, train_idx)

    # ایجاد DataLoaderها
    train_loader = DataLoader(
        train_subset,
        batch_size=config.BATCH_SIZE,
        sampler=train_sampler,
        num_workers=2
    )
    val_loader = DataLoader(val_subset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2)

    # ایجاد مدل
    model = AdvancedResNet(num_classes=config.NUM_CLASSES, dropout=config.DROPOUT)
    model = model.to(device)

    # نمایش معماری مدل
    print(f"\nModel Architecture:")
    print(model)

    # تنظیم loss function و optimizer
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.AdamW(
        model.parameters(),
        lr=config.LEARNING_RATE,
        weight_decay=config.WEIGHT_DECAY,
        betas=(0.9, 0.999)
    )

    # Scheduler پیشرفته‌تر
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=10,
        T_mult=2,
        eta_min=1e-6
    )

    # آموزش مدل
    print("\n🎯 Starting Training...")
    best_val_accuracy = 0
    no_improve = 0

    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for epoch in range(config.NUM_EPOCHS):
        print(f'\n{"="*50}')
        print(f'Epoch {epoch+1}/{config.NUM_EPOCHS}')
        print(f'Current LR: {optimizer.param_groups[0]["lr"]:.6f}')

        # آموزش
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)

        # ارزیابی
        val_loss, val_acc, val_auc = validate_epoch(model, val_loader, criterion, device)

        # به‌روزرسانی scheduler
        scheduler.step()

        # ذخیره تاریخچه
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        print(f'\nEpoch {epoch+1} Summary:')
        print(f'Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}% | AUC: {val_auc:.4f}')

        # ذخیره بهترین مدل
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': val_acc,
                'val_auc': val_auc
            }, 'best_model.pth')
            no_improve = 0
            print(f'✅ New best model saved! Val Accuracy: {val_acc:.2f}%')
        else:
            no_improve += 1
            print(f'No improvement for {no_improve} epochs')

        # Early Stopping
        if no_improve >= config.PATIENCE:
            print(f'\n🛑 Early stopping at epoch {epoch+1}')
            break

    # پلات تاریخچه آموزش
    plot_training_history(train_losses, val_losses, train_accs, val_accs)

    # ارزیابی نهایی روی تست‌ست
    print("\n🧪 Final Evaluation on Test Set...")

    # بارگذاری بهترین مدل
    checkpoint = torch.load('best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])

    model.eval()
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc='Testing'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            probs = torch.softmax(outputs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # محاسبه متریک‌ها
    test_acc = 100 * np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
    test_auc = roc_auc_score(all_labels, np.array(all_probs)[:, 1])

    print(f'\n📊 Final Test Results:')
    print(f'Test Accuracy: {test_acc:.2f}%')
    print(f'Test AUC: {test_auc:.4f}')

    print('\n📈 Classification Report:')
    print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

    print('\n🎯 Confusion Matrix:')
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=test_dataset.classes,
                yticklabels=test_dataset.classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    # نمایش نمونه‌هایی از پیش‌بینی‌ها
    print(f"\n💡 Key Insights:")
    print(f"- Best Validation Accuracy: {best_val_accuracy:.2f}%")
    print(f"- Test Accuracy: {test_acc:.2f}%")
    print(f"- Test AUC: {test_auc:.4f}")
    print(f"- Gap between Val and Test: {abs(best_val_accuracy - test_acc):.2f}%")

    if abs(best_val_accuracy - test_acc) < 5:
        print("✅ Good generalization! Small gap between validation and test.")
    else:
        print("⚠️  Potential overfitting! Consider more regularization.")

if __name__ == "__main__":
    main()

🚀 Starting Advanced Lung Cancer Detection Training...
Using device: cpu
Train path: /content/lung-cancer-x-ray/train
Test path: /content/lung-cancer-x-ray/test
Train classes: ['Cancer', 'NORMAL']
Test classes: ['Cancer', 'NORMAL']
Train dataset size: 5216
Test dataset size: 624


KeyboardInterrupt: 

In [ ]:
#import کتابخانه های لازم

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit


#تعریف هایپرپارامترها

BATCH_SIZE = 32
DROPOUT = 0.5
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
IMG_SIZE = 224
PATIENCE = 5
N_FEATURES = 100

#data augmentation

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#بخش بارگیری داده ها

def setup_dataset():
    base_path = '/content/chest_xray'
    if not os.path.exists(base_path):
        print("Downloading dataset...")
        os.makedirs('/content', exist_ok=True)
        os.system('kaggle datasets download -d paultimothymooney/chest-xray-pneumonia')
        os.system('unzip -q chest-xray-pneumonia.zip')

    train_path = os.path.join(base_path, 'train')
    test_path = os.path.join(base_path, 'test')
    return train_path, test_path

train_path, test_path = setup_dataset()

#بخش های آموزشی دیتا ست

train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=val_transform)


targets = np.array([label for _, label in train_dataset])
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(np.zeros(len(targets)), targets))

train_subset = Subset(train_dataset, train_idx)
val_subset = Subset(train_dataset, val_idx)

#محاسبه وزن کلاس ها برای handling imbalance

class_counts = np.bincount(targets[train_idx])
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float32)

#ایجاد بخش های Dataloader

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

#تعریف مدل شبکه عصبی

def create_model():
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(DROPOUT),
        nn.Linear(num_ftrs, 2))
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#تابع ارزیابی مدل برای gwo

def evaluate_model(feature_mask):
    model = create_model().to(device)


    for name, param in model.named_parameters():
        if 'fc' in name and 'weight' in name:

            mask = torch.tensor(feature_mask, dtype=torch.float32).to(device).view(1, -1)
            param.data = param.data * mask

    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)


    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        break


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

#تابع هدف برای الگوریتم gwo

def objective_function(position):

    feature_mask = (position > 0.5).astype(float)


    if np.sum(feature_mask) == 0:
        return 100


    accuracy = evaluate_model(feature_mask)


    error = 1.0 - accuracy


    penalty = 0.001 * np.sum(feature_mask)

    return error + penalty

#پیاده سازی الگوریتم گرگ خاکستری

class GWO:
    def __init__(self, obj_function, dim, n_agents, max_iter, lb, ub):
        self.obj_function = obj_function
        self.dim = dim
        self.n_agents = n_agents
        self.max_iter = max_iter
        self.lb = lb
        self.ub = ub


        self.positions = np.random.uniform(lb, ub, (n_agents, dim))


        self.alpha_pos = np.zeros(dim)
        self.alpha_score = float("inf")

        self.beta_pos = np.zeros(dim)
        self.beta_score = float("inf")

        self.delta_pos = np.zeros(dim)
        self.delta_score = float("inf")


        self.best_scores = []

    def optimize(self):
        for iter in range(self.max_iter):
            for i in range(self.n_agents):

                self.positions[i, :] = np.clip(self.positions[i, :], self.lb, self.ub)


                fitness = self.obj_function(self.positions[i, :])


                if fitness < self.alpha_score:
                    self.alpha_score = fitness
                    self.alpha_pos = self.positions[i, :].copy()
                elif fitness < self.beta_score:
                    self.beta_score = fitness
                    self.beta_pos = self.positions[i, :].copy()
                elif fitness < self.delta_score:
                    self.delta_score = fitness
                    self.delta_pos = self.positions[i, :].copy()


            a = 2 - iter * (2 / self.max_iter)


            for i in range(self.n_agents):
                for j in range(self.dim):

                    A1 = 2 * a * np.random.random() - a
                    C1 = 2 * np.random.random()

                    A2 = 2 * a * np.random.random() - a
                    C2 = 2 * np.random.random()

                    A3 = 2 * a * np.random.random() - a
                    C3 = 2 * np.random.random()


                    D_alpha = abs(C1 * self.alpha_pos[j] - self.positions[i, j])
                    X1 = self.alpha_pos[j] - A1 * D_alpha

                    D_beta = abs(C2 * self.beta_pos[j] - self.positions[i, j])
                    X2 = self.beta_pos[j] - A2 * D_beta

                    D_delta = abs(C3 * self.delta_pos[j] - self.positions[i, j])
                    X3 = self.delta_pos[j] - A3 * D_delta


                    self.positions[i, j] = (X1 + X2 + X3) / 3


            self.best_scores.append(self.alpha_score)


            print(f"Iteration {iter+1}/{self.max_iter}, Best Error: {self.alpha_score:.4f}, "
                  f"Best Accuracy: {1 - self.alpha_score:.4f}")

        return self.alpha_pos, 1 - self.alpha_score

#اجرای الگوریتم گرگ خاکستری برای انتخاب ویژگی

dim = 512
n_agents = 10
max_iter = 20
lb = 0
ub = 1

print("Starting GWO optimization...")
gwo = GWO(objective_function, dim, n_agents, max_iter, lb, ub)
best_position, best_accuracy = gwo.optimize()

#آموزش مدل نهایی با ویژگی های انتخاب شده

best_feature_mask = (best_position > 0.5).astype(float)
print(f"\nSelected {np.sum(best_feature_mask)} features out of {dim}")


print("\nTraining final model with selected features...")
final_model = create_model().to(device)


for name, param in final_model.named_parameters():
    if 'fc' in name and 'weight' in name:
        mask = torch.tensor(best_feature_mask, dtype=torch.float32).to(device).view(1, -1)
        param.data = param.data * mask


criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.AdamW(final_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2)

best_val_accuracy = 0
no_improve = 0

for epoch in range(NUM_EPOCHS):
    final_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total


    final_model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = final_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total

    scheduler.step(val_acc)

    print(f'\nEpoch {epoch+1}/{NUM_EPOCHS}:')
    print(f'Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%')

    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(final_model.state_dict(), 'best_model.pth')
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            print(f'\nEarly stopping at epoch {epoch+1}')
            break

#ارزیابی مدل روی داده های تست

final_model.load_state_dict(torch.load('best_model.pth'))
final_model.eval()

all_labels = []
all_preds = []
test_correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

#ارزیابی نهایی مدل

test_acc = 100 * test_correct / len(test_dataset)
print(f'\nFinal Test Accuracy: {test_acc:.2f}%')
print('\nClassification Report:')
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))
print('\nConfusion Matrix:')
print(confusion_matrix(all_labels, all_preds))

Starting GWO optimization...
Iteration 1/20, Best Error: 0.3920, Best Accuracy: 0.6080
Iteration 2/20, Best Error: 0.3636, Best Accuracy: 0.6364
Iteration 3/20, Best Error: 0.3636, Best Accuracy: 0.6364
Iteration 4/20, Best Error: 0.3521, Best Accuracy: 0.6479
Iteration 5/20, Best Error: 0.3521, Best Accuracy: 0.6479
Iteration 6/20, Best Error: 0.3046, Best Accuracy: 0.6954
Iteration 7/20, Best Error: 0.3046, Best Accuracy: 0.6954
Iteration 8/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 9/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 10/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 11/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 12/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 13/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 14/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 15/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 16/20, Best Error: 0.2715, Best Accuracy: 0.7285
Iteration 17/20, Best Error: 0.2715,

In [ ]:
# import کتابخانه های لازم

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
import time


# تعریف هایپرپارامترها

BATCH_SIZE = 32
DROPOUT = 0.5
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
IMG_SIZE = 224
PATIENCE = 5
N_FEATURES = 100

# data augmentation

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# بخش بارگیری داده ها

def setup_dataset():
    base_path = '/content/chest_xray'
    if not os.path.exists(base_path):
        print("Downloading dataset...")
        os.makedirs('/content', exist_ok=True)
        os.system('kaggle datasets download -d paultimothymooney/chest-xray-pneumonia')
        os.system('unzip -q chest-xray-pneumonia.zip')

    train_path = os.path.join(base_path, 'train')
    test_path = os.path.join(base_path, 'test')
    return train_path, test_path

train_path, test_path = setup_dataset()

# بخش های آموزشی دیتا ست

train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=val_transform)


targets = np.array([label for _, label in train_dataset])
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(np.zeros(len(targets)), targets))

train_subset = Subset(train_dataset, train_idx)
val_subset = Subset(train_dataset, val_idx)

# محاسبه وزن کلاس ها برای handling imbalance

class_counts = np.bincount(targets[train_idx])
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# ایجاد بخش های Dataloader

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# تعریف مدل شبکه عصبی

def create_model():
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(DROPOUT),
        nn.Linear(num_ftrs, 2))
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تابع ارزیابی مدل برای GWO - بهبود یافته
def evaluate_model(feature_mask):
    model = create_model().to(device)

    # اعمال ماسک ویژگی‌ها روی لایه fc
    for name, param in model.named_parameters():
        if 'fc.1.weight' in name:  # لایه linear اصلی
            mask = torch.tensor(feature_mask, dtype=torch.float32).to(device)
            # اطمینان از تطابق ابعاد
            if mask.shape[0] == param.data.shape[1]:
                mask = mask.view(1, -1)
                param.data = param.data * mask

    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    # آموزش کوتاه برای ارزیابی
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        if i >= 3:  # فقط 3 batch برای ارزیابی سریع
            break
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # ارزیابی
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total if total > 0 else 0
    return accuracy

# تابع هدف برای الگوریتم GWO - بهبود یافته
def objective_function(position):
    # تبدیل موقعیت به ماسک باینری
    feature_mask = (position > 0.5).astype(float)

    # اگر هیچ ویژگی‌ای انتخاب نشده باشد، جریمه سنگین
    if np.sum(feature_mask) == 0:
        return 100.0  # جریمه سنگین

    try:
        accuracy = evaluate_model(feature_mask)
        error = 1.0 - accuracy

        # جریمه برای تعداد ویژگی‌های انتخاب شده (ترجیح انتخاب ویژگی‌های کمتر)
        penalty = 0.01 * (np.sum(feature_mask) / len(feature_mask))

        return error + penalty

    except Exception as e:
        print(f"Error in evaluation: {e}")
        return 100.0  # در صورت خطا، مقدار بد برگردان

# پیاده سازی الگوریتم گرگ خاکستری - بهبود یافته
class GWO:
    def __init__(self, obj_function, dim, n_agents, max_iter, lb, ub):
        self.obj_function = obj_function
        self.dim = dim
        self.n_agents = n_agents
        self.max_iter = max_iter
        self.lb = lb
        self.ub = ub

        # مقداردهی اولیه موقعیت‌ها
        self.positions = np.random.uniform(lb, ub, (n_agents, dim))

        # مقداردهی اولیه رهبران
        self.alpha_pos = np.zeros(dim)
        self.alpha_score = float("inf")

        self.beta_pos = np.zeros(dim)
        self.beta_score = float("inf")

        self.delta_pos = np.zeros(dim)
        self.delta_score = float("inf")

        self.best_scores = []
        self.execution_times = []

    def optimize(self):
        print(f"Starting GWO optimization with {self.n_agents} agents and {self.max_iter} iterations...")

        for iter in range(self.max_iter):
            start_time = time.time()

            a = 2.0 - iter * (2.0 / self.max_iter)  # ضریب a linearly decreases from 2 to 0

            for i in range(self.n_agents):
                # اعمال محدودیت‌ها روی موقعیت
                self.positions[i, :] = np.clip(self.positions[i, :], self.lb, self.ub)

                # محاسبه fitness
                fitness = self.obj_function(self.positions[i, :])

                # به روزرسانی رهبران
                if fitness < self.alpha_score:
                    self.alpha_score = fitness
                    self.alpha_pos = self.positions[i, :].copy()
                elif fitness < self.beta_score:
                    self.beta_score = fitness
                    self.beta_pos = self.positions[i, :].copy()
                elif fitness < self.delta_score:
                    self.delta_score = fitness
                    self.delta_pos = self.positions[i, :].copy()

            # به روزرسانی موقعیت تمامی عامل‌ها
            for i in range(self.n_agents):
                for j in range(self.dim):
                    # بروزرسانی بر اساس alpha
                    r1 = np.random.random()
                    r2 = np.random.random()
                    A1 = 2.0 * a * r1 - a
                    C1 = 2.0 * r2
                    D_alpha = abs(C1 * self.alpha_pos[j] - self.positions[i, j])
                    X1 = self.alpha_pos[j] - A1 * D_alpha

                    # بروزرسانی بر اساس beta
                    r1 = np.random.random()
                    r2 = np.random.random()
                    A2 = 2.0 * a * r1 - a
                    C2 = 2.0 * r2
                    D_beta = abs(C2 * self.beta_pos[j] - self.positions[i, j])
                    X2 = self.beta_pos[j] - A2 * D_beta

                    # بروزرسانی بر اساس delta
                    r1 = np.random.random()
                    r2 = np.random.random()
                    A3 = 2.0 * a * r1 - a
                    C3 = 2.0 * r2
                    D_delta = abs(C3 * self.delta_pos[j] - self.positions[i, j])
                    X3 = self.delta_pos[j] - A3 * D_delta

                    # میانگین موقعیت‌ها
                    self.positions[i, j] = (X1 + X2 + X3) / 3.0

            end_time = time.time()
            iteration_time = end_time - start_time
            self.execution_times.append(iteration_time)
            self.best_scores.append(self.alpha_score)

            # محاسبه تعداد ویژگی‌های انتخاب شده
            selected_features = np.sum(self.alpha_pos > 0.5)

            print(f"Iteration {iter+1}/{self.max_iter}, Best Error: {self.alpha_score:.4f}, "
                  f"Best Accuracy: {1 - self.alpha_score:.4f}, "
                  f"Selected Features: {selected_features}/{self.dim}, "
                  f"Time: {iteration_time:.2f}s")

        return self.alpha_pos, 1 - self.alpha_score

# اجرای الگوریتم گرگ خاکستری برای انتخاب ویژگی
dim = 512  # تعداد ویژگی‌ها (مطابق با خروجی resnet18)
n_agents = 5  # کاهش تعداد عامل‌ها برای سرعت بیشتر
max_iter = 5   # کاهش تعداد iteration‌ها
lb = 0
ub = 1

print("Starting GWO optimization for feature selection...")
gwo = GWO(objective_function, dim, n_agents, max_iter, lb, ub)
best_position, best_accuracy = gwo.optimize()

# آموزش مدل نهایی با ویژگی های انتخاب شده
best_feature_mask = (best_position > 0.5).astype(float)
selected_features = np.sum(best_feature_mask)
print(f"\nSelected {selected_features} features out of {dim}")
print(f"Feature mask: {best_feature_mask}")

print("\nTraining final model with selected features...")
final_model = create_model().to(device)

# اعمال ماسک ویژگی‌های انتخاب شده
for name, param in final_model.named_parameters():
    if 'fc.1.weight' in name:  # لایه linear اصلی
        mask = torch.tensor(best_feature_mask, dtype=torch.float32).to(device)
        mask = mask.view(1, -1)
        param.data = param.data * mask

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(final_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

best_val_accuracy = 0
no_improve = 0

for epoch in range(NUM_EPOCHS):
    final_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    # ارزیابی
    final_model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = final_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total

    scheduler.step(val_acc)

    print(f'\nEpoch {epoch+1}/{NUM_EPOCHS}:')
    print(f'Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%')

    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(final_model.state_dict(), 'best_model.pth')
        no_improve = 0
        print('Validation accuracy improved! Model saved.')
    else:
        no_improve += 1
        print(f'No improvement for {no_improve} epochs.')
        if no_improve >= PATIENCE:
            print(f'\nEarly stopping at epoch {epoch+1}')
            break

# ارزیابی مدل روی داده های تست
final_model.load_state_dict(torch.load('best_model.pth'))
final_model.eval()

all_labels = []
all_preds = []
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# ارزیابی نهایی مدل
test_acc = 100 * test_correct / test_total
print(f'\nFinal Test Accuracy: {test_acc:.2f}%')
print(f'Test Samples: {test_total}')
print('\nClassification Report:')
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))
print('\nConfusion Matrix:')
print(confusion_matrix(all_labels, all_preds))

# نمایش نتایج GWO
print(f'\nGWO Results:')
print(f'Best Accuracy: {best_accuracy:.4f}')
print(f'Best Position: {best_position}')
print(f'Selected Features: {selected_features}/{dim}')
print(f'Average iteration time: {np.mean(gwo.execution_times):.2f}s')

Starting GWO optimization for feature selection...
Starting GWO optimization with 5 agents and 5 iterations...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 101MB/s]


Iteration 1/5, Best Error: 0.0799, Best Accuracy: 0.9201, Selected Features: 265/512, Time: 430.98s
Iteration 2/5, Best Error: 0.0799, Best Accuracy: 0.9201, Selected Features: 265/512, Time: 426.71s
Iteration 3/5, Best Error: 0.0799, Best Accuracy: 0.9201, Selected Features: 265/512, Time: 422.32s
Iteration 4/5, Best Error: 0.0799, Best Accuracy: 0.9201, Selected Features: 265/512, Time: 423.87s
Iteration 5/5, Best Error: 0.0593, Best Accuracy: 0.9407, Selected Features: 192/512, Time: 426.40s

Selected 192.0 features out of 512
Feature mask: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1.
 0. 1. 1. 1.